In [4]:
pip install kagglehub scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [36]:
import os
import shutil
import kagglehub
from pathlib import Path
from ultralytics import YOLO, SETTINGS
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv

In [37]:
load_dotenv()

True

In [38]:
# Configuration
DATASET_NAME = "sanidhyak/human-face-emotions"
CLASSES = ["happy", "sad", "angry"]
IMG_SIZE = 640
BATCH_SIZE = 16
EPOCHS = 100

In [39]:
# 2. Kaggle Authentication Setup
os.environ["KAGGLE_USERNAME"] = os.getenv("KAGGLE_USERNAME")
os.environ["KAGGLE_KEY"] = os.getenv("KAGGLE_KEY")

In [40]:
def prepare_dataset():
    try:
        dataset_path = Path(kagglehub.dataset_download(DATASET_NAME))
        print(f"Dataset downloaded to: {dataset_path}")

        # The 'data' directory contains class folders
        data_dir = dataset_path / "data"
        if not data_dir.exists():
            raise FileNotFoundError(f"'data' folder not found in {dataset_path}")

        base_path = Path("datasets/emotion").resolve()
        (base_path/"images/train").mkdir(parents=True, exist_ok=True)
        (base_path/"images/val").mkdir(parents=True, exist_ok=True)
        (base_path/"labels/train").mkdir(parents=True, exist_ok=True)
        (base_path/"labels/val").mkdir(parents=True, exist_ok=True)

        total_images = 0
        valid_classes = {c.lower(): i for i, c in enumerate(CLASSES)}

        for class_dir in data_dir.iterdir():
            if not class_dir.is_dir():
                continue
            class_name = class_dir.name.lower()
            if class_name not in valid_classes:
                print(f"Skipping unexpected directory: {class_dir.name}")
                continue

            images = [img for img in class_dir.iterdir() if img.suffix.lower() in [".jpg", ".jpeg", ".png"] and img.is_file()]
            if not images:
                print(f"No images found in {class_dir}")
                continue

            train_imgs, val_imgs = train_test_split(
                images, 
                test_size=0.2, 
                random_state=42,
                shuffle=True
            )

            class_id = valid_classes[class_name]
            copied = 0

            for split, imgs in [("train", train_imgs), ("val", val_imgs)]:
                for img in imgs:
                    dest_img = base_path/"images"/split/img.name
                    dest_label = base_path/"labels"/split/f"{img.stem}.txt"
                    shutil.copy(img, dest_img)
                    with open(dest_label, "w") as f:
                        f.write(f"{class_id} 0.5 0.5 1.0 1.0")
                    copied += 1

            total_images += copied
            print(f"Processed {copied} images from {class_dir.name}")

        if total_images == 0:
            raise RuntimeError("No valid images found after processing")

        yaml_content = f"""path: {base_path}
train: images/train
val: images/val
names: {CLASSES}
"""
        yaml_path = base_path/"dataset.yaml"
        yaml_path.write_text(yaml_content)
        print(f"Dataset prepared successfully with {total_images} images")
        return True

    except Exception as e:
        print(f"Dataset preparation failed: {str(e)}")
        return False

In [45]:
def train_model():
    try:
        # 1. Configure Ultralytics settings
        SETTINGS.update({
            'datasets_dir': str(Path.cwd()),
            'weights_dir': str(Path.cwd()/"models"),
            'runs_dir': str(Path.cwd()/"results")
        })
        
        # 2. Load model with verification
        weights_path = Path('yolov8n.pt').resolve()
        if not weights_path.exists():
            raise FileNotFoundError(f"Pretrained weights not found at {weights_path}")
            
        model = YOLO(str(weights_path))  # Convert to string for YOLO compatibility
        
        # 3. Get absolute path to dataset.yaml
        yaml_path = Path("datasets/emotion/dataset.yaml").resolve()
        if not yaml_path.exists():
            raise FileNotFoundError(f"Dataset config not found at {yaml_path}")

        # 4. Start training with full diagnostics
        results = model.train(
            data=str(yaml_path),
            epochs=30,             # ↓ Lower epochs
            imgsz=416,             # ↓ Smaller image size
            batch=BATCH_SIZE,
            name='emotion_yolov8n',
            exist_ok=True,
            pretrained=True,
            optimizer='AdamW',
            lr0=0.001,
            augment=False,         # ↓ Faster training (turn on later for final training)
            verbose=True,
            plots=False
        )
        
        # 5. Export and verify model
        export_path = model.export(format='onnx')
        if not Path(export_path).exists():
            raise RuntimeError(f"Model export failed at {export_path}")
            
        return True

    except Exception as e:
        print(f"Training failed: {str(e)}")
        return False

In [ ]:
if __name__ == "__main__":
    if prepare_dataset():
        print("\nStarting training process...")
        if train_model():
            print("\nTraining completed successfully")
            model = YOLO('runs/detect/emotion_yolov8n/weights/best.pt')
            print("\nModel summary:")
            print(model.info())
        else:
            print("\nTraining failed")
    else:
        print("\nDataset preparation failed")

Dataset downloaded to: C:\Users\Preetish\.cache\kagglehub\datasets\sanidhyak\human-face-emotions\versions\1
Processed 86 images from Angry
Processed 100 images from Happy
Processed 78 images from Sad
Dataset prepared successfully with 264 images

Starting training process...
New https://pypi.org/project/ultralytics/8.3.142 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.138  Python-3.12.7 torch-2.7.0+cpu CPU (Intel Core(TM) i5-1035G1 1.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\VS Code Projects\Human-Emotion-Recognition\datasets\emotion\dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1

train: Scanning D:\VS Code Projects\Human-Emotion-Recognition\datasets\emotion\labels\train.cache... 208 images, 0 backgrounds, 0 corrupt: 100%|██████████| 208/208 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 10.28.5 MB/s, size: 107.8 KB)



d:\Anaconda Installation\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning D:\VS Code Projects\Human-Emotion-Recognition\datasets\emotion\labels\val... 54 images, 0 backgrounds, 0 corrupt: 100%|██████████| 54/54 [00:00<00:00, 92.78it/s] 

val: D:\VS Code Projects\Human-Emotion-Recognition\datasets\emotion\images\val\p24783774_b_v13_ab.jpg: corrupt JPEG restored and saved
val: New cache created: D:\VS Code Projects\Human-Emotion-Recognition\datasets\emotion\labels\val.cache
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to runs\detect\emotion_yolov8n
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/30         0G     0.2785      1.247     0.9394         54        416: 100%|██████████| 13/13 [00:53<00:00,  4.12s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:06<00:00,  3.09s/it]

                   all         54         54      0.444      0.549      0.495      0.481



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/30         0G     0.2625      1.236     0.9308         54        416:  38%|███▊      | 5/13 [00:17<00:30,  3.77s/it]